In [ ]:
import glob
import json
import datetime as dt
import pandas as pd
import os

In [ ]:
#### SETUP ####
wdir = "/media/windows/projects/kriging/Scripts/unibz/test_python/"
config_path = wdir + "etc/config/"
input_path = wdir + "input/"

In [ ]:
inputs = glob.glob( input_path + "*.json" )

In [ ]:
for input in inputs:
    # print(input)

    setup_file = open(input)
    setup_sim = json.load(setup_file)
    variable = setup_sim["variable"]

    # coordinates = coordinates.format(
    #     east=east, north=north, elevation=elevation)
    # config_file = open(config_path + "temperature.json")
    # file = open(pathout+"subcells.in", "w+")
    # [file.writelines(el + "\n") for el in [header, cell_metadata, coordinates]]

    if variable == "temperature":
        config_file = open(config_path + "temperature.json")
    elif variable == "precipitation":
        config_file = open(config_path + "precipitation.json")

    params = json.load(config_file)

    simulation_type = setup_sim["simulation_type"]

    basin = setup_sim["basin"]
    secs_timestep = setup_sim["secs_timestep"]
    number_of_stations = setup_sim["number_of_stations"]
    kriging_type = setup_sim["kriging_type"]
    kriging_model = params["kriging_model"]
    kriging_correction = setup_sim["kriging_correction"]

    number_of_cells = len(setup_sim["cells"])
    # print("Number of points: " + str(number_of_cells))

    start_datetime_str = setup_sim["start_datetime"]
    start_datetime = dt.datetime.strptime(
        start_datetime_str, '%Y-%m-%d %H:%M:%S')
    end_datetime_str = setup_sim["end_datetime"]
    end_datetime = dt.datetime.strptime(end_datetime_str, '%Y-%m-%d %H:%M:%S')
    dates = pd.date_range(start_datetime, end_datetime, freq='h')
    steps = len(dates)

    weather_data_path = wdir + setup_sim["weather_data_path"]

    path_kriging_inputs = wdir + basin + "/input/"
    path_kriging_outputs = wdir + basin + "/output/"

    # OUTPUT DIRECTORY
    output_dir = wdir + setup_sim['output_path'] + variable + "/" + simulation_type + \
        "/" + kriging_type + "/" + kriging_correction + "/"
    output_path_stats = output_dir + "stats/"

    outputs = glob.glob(output_path_stats + "*.txt")

    # create stats
    sum_hourly_rmse = 0.0
    sum_daily_rmse = 0.0
    sum_monthly_rmse = 0.0

    sum_hourly_mae = 0.0
    sum_daily_mae = 0.0
    sum_monthly_mae = 0.0

    sum_hourly_r2 = 0.0
    sum_daily_r2 = 0.0
    sum_monthly_r2 = 0.0

    for out in outputs:

        # print(os.path.basename(out).split("_"))

        out_stats = pd.read_csv(out, index_col=0)
        sum_hourly_rmse = sum_hourly_rmse + \
            float(out_stats.loc["hourly"]["RMSE"])
        sum_daily_rmse = sum_daily_rmse + float(out_stats.loc["daily"]["RMSE"])
        sum_monthly_rmse = sum_monthly_rmse + \
            float(out_stats.loc["monthly"]["RMSE"])

        sum_hourly_mae = sum_hourly_mae + float(out_stats.loc["hourly"]["MAE"])
        sum_daily_mae = sum_daily_mae + float(out_stats.loc["daily"]["MAE"])
        sum_monthly_mae = sum_monthly_mae + \
            float(out_stats.loc["monthly"]["MAE"])

        sum_hourly_r2 = sum_hourly_r2 + float(out_stats.loc["hourly"]["R2"])
        sum_daily_r2 = sum_daily_r2 + float(out_stats.loc["daily"]["R2"])
        sum_monthly_r2 = sum_monthly_r2 + float(out_stats.loc["monthly"]["R2"])

    rmse_hourly_mean = sum_hourly_rmse / len(outputs)
    rmse_daily_mean = sum_daily_rmse / len(outputs)
    rmse_monthly_mean = sum_monthly_rmse / len(outputs)

    mae_hourly_mean = sum_hourly_mae / len(outputs)
    mae_daily_mean = sum_daily_mae / len(outputs)
    mae_monthly_mean = sum_monthly_mae / len(outputs)

    r2_hourly_mean = sum_hourly_r2 / len(outputs)
    r2_daily_mean = sum_daily_r2 / len(outputs)
    r2_monthly_mean = sum_monthly_r2 / len(outputs)

    data = [["hourly", rmse_hourly_mean, mae_hourly_mean, r2_hourly_mean],
            ["daily", rmse_daily_mean, mae_daily_mean, r2_daily_mean],
            ["monthly", rmse_monthly_mean, mae_monthly_mean, r2_monthly_mean]]

    output_stats = pd.DataFrame(data, columns=["timestep", "RMSE", "MAE", "R2"])
    output_stats.set_index("timestep", inplace=True)

    output_stats.to_csv( output_path_stats + "00_statistics_mean.txt" )

In [ ]:
# data_stats = pd.DataFrame( columns=['elevation', 'error'] )

In [ ]:
# data_stats['elevation'] = elevation_array
# data_stats['error'] = error_array

In [ ]:
# data_stats